# Dimensionality Reduction Using Feature Extraction

Feature extraction: how to reduce the dimensionality of our feature matrix by
creating new features with (ideally) similar ability to train quality models but with
significantly fewer dimensions.

- One downside of the feature extraction techniques is that the new features
we generate will not be interpretable by humans. They will contain as much or nearly
as much ability to train our models, but will appear to the human eye as a collection
of random numbers.
<!-- 2018, Albon, Machine Learning with Python Cookbook. Cap 9-10 -->

## Reducing Features Using Principal Components

Principal component analysis (PCA) projects observations onto the (hopefully fewer) principal components of the feature matrix that retain the most variance. PCA is an unsupervised technique, meaning that it does not use the information from the target vector and instead only considers the feature matrix.


![pca](https://sebastianraschka.com/images/faq/lda-vs-pca/pca.png)

https://scikit-learn.org/stable/modules/generated/sklearn.decomposition.PCA.html#sklearn.decomposition.PCA

In [ ]:
## Given a set of features, you want to reduce the number of features while retaining 
## the variance in the data.

# Load libraries
from sklearn.preprocessing import StandardScaler
from sklearn.decomposition import PCA
from sklearn import datasets

# Load the data
#digitos del cero al nueve a mano
digits = datasets.load_digits()

# Standardize the feature matrix
features = StandardScaler().fit_transform(digits.data)

# Create a PCA that will retain 99% of variance
pca = PCA(n_components=0.99, whiten=True)

# Conduct PCA
features_pca = pca.fit_transform(features)

# Show results
print("Original number of features:", features.shape[1])
print("Reduced number of features:", features_pca.shape[1])

Original number of features: 64
Reduced number of features: 16


PCA is implemented in scikit-learn using the `pca` method. `n_components` has two operations, depending on the argument provided. 
- If the argument is greater than 1, `n_components` will return that many features. 
- If the argument to n_components is between 0 and 1, `pca` returns the minimum amount of features that retain that much variance. 

It is common to use values of 0.95 and 0.99, meaning 95%
and 99% of the variance of the original features has been retained, respectively.

`whiten=True` transforms the values of each principal component so that they have zero mean and unit variance. 

Another parameter and argument is
`svd_solver="randomized"`, which implements a stochastic algorithm to find the first
principal components in often significantly less time.

## Reducing Features When Data Is Linearly Inseparable

If your data is not linearly separable (e.g., you
can only separate classes using a curved decision boundary), the linear transformation will not work as well. In our solution we used scikit-learn’s `make_circles` to generate a simulated dataset with a target vector of two classes and two features.

Kernels allow us to project the linearly inseparable data into a higher dimension where it is linearly separable; this is called the kernel trick. There are a number of kernels we can use in scikit-learn’s `kernelPCA`, specified using the kernel parameter. A common kernel to use is the Gaussian radial basis function kernel `rbf`, but other options are the polynomial kernel
(`poly`) and sigmoid kernel (`sigmoid`). We can even specify a linear projection (`linear`), which will produce the same results as standard PCA.

https://scikit-learn.org/stable/modules/generated/sklearn.decomposition.KernelPCA.html#sklearn.decomposition.KernelPCA

In [ ]:
## You suspect you have linearly inseparable data and want to reduce 
## the dimensions.

# Load libraries
from sklearn.decomposition import PCA, KernelPCA
from sklearn.datasets import make_circles

# Create linearly inseparable data
features, _ = make_circles(n_samples=1000, random_state=1, noise=0.1, factor=0.1)

# Apply kernal PCA with radius basis function (RBF) kernel
kpca = KernelPCA(kernel="rbf", gamma=15, n_components=1)
features_kpca = kpca.fit_transform(features)
print("Original number of features:", features.shape[1])
print("Reduced number of features:", features_kpca.shape[1])

Original number of features: 2
Reduced number of features: 1


## Reducing Features by Maximizing Class Separability

Linear discriminant analysis (LDA) is a classification that is also a popular technique for dimensionality reduction.
LDA works similarly to principal component analysis (PCA) in that it projects our
feature space onto a lower-dimensional space. However, in PCA we were only interested
in the component axes that maximize the variance in the data, while in LDA we
have the additional goal of maximizing the differences between classes.

![lda](https://sebastianraschka.com/images/faq/lda-vs-pca/lda.png)


https://scikit-learn.org/stable/auto_examples/decomposition/plot_pca_vs_lda.html#sphx-glr-auto-examples-decomposition-plot-pca-vs-lda-py

In [ ]:
## You want to reduce the features to be used by a classifier.

# Load libraries
from sklearn import datasets
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis

# Load Iris flower dataset:
iris = datasets.load_iris()
features = iris.data
target = iris.target

# Create and run an LDA, then use it to transform the features
lda = LinearDiscriminantAnalysis(n_components=1)
features_lda = lda.fit(features, target).transform(features)

# Print the number of features
print("Original number of features:", features.shape[1])
print("Reduced number of features:", features_lda.shape[1])

Original number of features: 4
Reduced number of features: 1


We can use `explained_variance_ratio_` to view the amount of variance explained by each component.

In [ ]:
lda.explained_variance_ratio_

array([0.9912126])

Specifically, we can run `LinearDiscriminantAnalysis` with `n_components` set to `None` to return the ratio of variance explained by every component feature, then calculate
how many components are required to get above some threshold of variance explained (often 0.95 or 0.99):

In [ ]:
# Create and run LDA
lda = LinearDiscriminantAnalysis(n_components=None)
features_lda = lda.fit(features, target)

# Create array of explained variance ratios
lda_var_ratios = lda.explained_variance_ratio_

# Create function
def select_n_components(var_ratio, goal_var: float) -> int:
    # Set initial variance explained so far
    total_variance = 0.0
    # Set initial number of features
    n_components = 0
    # For the explained variance of each feature:
    for explained_variance in var_ratio:
        # Add the explained variance to the total
        total_variance += explained_variance
        # Add one to the number of components
        n_components += 1
        # If we reach our goal level of explained variance
        if total_variance >= goal_var:
            # End the loop
            break
    # Return the number of components
    return n_components

# Run function
select_n_components(lda_var_ratios, 0.95)

1

## Reducing Features Using Matrix Factorization

**Non-negative matrix factorization** (NMF) is an unsupervised technique for linear dimensionality reduction that factorizes
(i.e., breaks up into multiple matrices whose product approximates the original matrix) the feature matrix into matrices representing the latent relationship between
observations and their features. Intuitively, NMF can reduce dimensionality because in matrix multiplication, the two factors (matrices being multiplied) can have significantly
fewer dimensions than the product matrix.

Formally, given a desired number of returned features, $r$, NMF factorizes our feature matrix such that:

<font size='5'> $ \mathbf{V} \approx \mathbf{WH} $ </font>

where $V$ is our $d \times n$ feature matrix (i.e., d features, n observations), $W$ is a $d \times r$,
and $H$ is an $r \times n$ matrix. 
- By adjusting the value of $r$ we can set the amount of dimensionality reduction desired.

In [ ]:
## You have a feature matrix of nonnegative values and want to reduce the dimensionality.

# Load libraries
from sklearn.decomposition import NMF
from sklearn import datasets

# Load the data
digits = datasets.load_digits()

# Load feature matrix
features = digits.data

# Create, fit, and apply NMF
nmf = NMF(n_components=10, random_state=1)
features_nmf = nmf.fit_transform(features)

# Show results
print("Original number of features:", features.shape[1])
print("Reduced number of features:", features_nmf.shape[1])

Original number of features: 64
Reduced number of features: 10


/usr/local/lib/python3.9/dist-packages/sklearn/decomposition/_nmf.py:1665: ConvergenceWarning: Maximum number of iterations 200 reached. Increase it to improve convergence.
  warnings.warn(


## Reducing Features on Sparse Data

*Truncated Singular Value Decomposition* (TSVD) is similar to PCA and in fact, PCA actually often uses non-truncated Singular
Value Decomposition (SVD) in one of its steps. In regular SVD, given $d$ features,
SVD will create factor matrices that are $d \times d$, whereas TSVD will return factors that
are $n \times n$, where $n$ is previously specified by a parameter. The practical advantage of
TSVD is that unlike PCA, it works on **sparse feature matrices**.

https://scikit-learn.org/stable/modules/generated/sklearn.decomposition.TruncatedSVD.html#sklearn.decomposition.TruncatedSVD

In [ ]:
## You have a sparse feature matrix and want to reduce the dimensionality.

# Load libraries
from sklearn.preprocessing import StandardScaler
from sklearn.decomposition import TruncatedSVD
from scipy.sparse import csr_matrix
from sklearn import datasets
import numpy as np

# Load the data
digits = datasets.load_digits()

# Standardize feature matrix
features = StandardScaler().fit_transform(digits.data)

# Make sparse matrix
features_sparse = csr_matrix(features)

# Create a TSVD
tsvd = TruncatedSVD(n_components=10)

# Conduct TSVD on sparse matrix
features_sparse_tsvd = tsvd.fit(features_sparse).transform(features_sparse)

# Show results
print("Original number of features:", features_sparse.shape[1])
print("Reduced number of features:", features_sparse_tsvd.shape[1])

Original number of features: 64
Reduced number of features: 10


TSVD provides us with the ratio of the original feature matrix’s variance
explained by each component

In [ ]:
# Sum of first three components' explained variance ratios
tsvd.explained_variance_ratio_[0:3].sum()

0.3003938539016248

We can automate the process by creating a function that runs TSVD with `n_components` set to one less than the number of original features and then calculate the number of components that explain a desired amount of the original data’s variance:

In [ ]:
# Create and run an TSVD with one less than number of features
tsvd = TruncatedSVD(n_components=features_sparse.shape[1]-1)
features_tsvd = tsvd.fit(features)

# List of explained variances
tsvd_var_ratios = tsvd.explained_variance_ratio_

# Create a function
def select_n_components(var_ratio, goal_var):
    # Set initial variance explained so far
    total_variance = 0.0
    # Set initial number of features
    n_components = 0
    # For the explained variance of each feature:
    for explained_variance in var_ratio:
        # Add the explained variance to the total
        total_variance += explained_variance
        # Add one to the number of components
        n_components += 1
        # If we reach our goal level of explained variance
        if total_variance >= goal_var:
            # End the loop
            break
    # Return the number of components
    return n_components

# Run function
select_n_components(tsvd_var_ratios, 0.95)

40

# Dimensionality Reduction Using Feature Selection

Feature selection: selecting high-quality, informative features and
dropping less useful features. There are three types of feature selection methods: filter, wrapper, and embedded. 

- Filter methods select the best features by examining their statistical properties. 
- Wrapper methods use trial and error to find the subset of features that produce models with the highest quality predictions. 
- Embedded methods select the best feature subset as part or as an extension of a learning algorithm’s training process.

## Thresholding Numerical Feature Variance

**Variance thresholding** (VT) is one of the most basic approaches to feature selection. It is motivated by the idea that features with low variance are likely less interesting (and
useful) than features with high variance. VT first calculates the variance of each feature:

<font size='5'> $ Var(x)=\frac{1}{n}\sum_{i=1}^{n}{(x_i-\mu)^2} $ </font>

where $x$ is the feature vector, $x_i$ is an individual feature value, and $\mu$ is that feature’s
mean value. Next, it drops all features whose variance does not meet that threshold.
There are two things to keep in mind when employing VT:
- the variance is not centered; that is, it is in the squared unit of the feature itself. Therefore, the VT will not work when feature sets contain different units (e.g., one feature is in years while a different feature is in dollars). 
- the variance threshold is selected manually, so we have to use our own judgment for a good value to select

In [ ]:
## You have a set of numerical features and want to remove those with low variance (i.e.,likely containing little information).
## S/ Select a subset of features with variances above a given threshold:

# Load libraries
from sklearn import datasets
from sklearn.feature_selection import VarianceThreshold

# import some data to play with
iris = datasets.load_iris()

# Create features and target
features = iris.data
target = iris.target

# Create thresholder
thresholder = VarianceThreshold(threshold=.5)

# Create high variance feature matrix
features_high_variance = thresholder.fit_transform(features)

print(features.shape)
print(features_high_variance.shape)

(150, 4)
(150, 3)


In [ ]:
# View high variance feature matrix
features_high_variance[0:3]

array([[5.1, 1.4, 0.2],
       [4.9, 1.4, 0.2],
       [4.7, 1.3, 0.2]])

We can see the variance for each feature using
variances

In [ ]:
# View variances
thresholder.fit(features).variances_

array([0.68112222, 0.18871289, 3.09550267, 0.57713289])

Finally, if the features have been standardized (to mean zero and unit variance), then for obvious reasons variance thresholding will not work correctly:

In [ ]:
# Load library
from sklearn.preprocessing import StandardScaler

# Standardize feature matrix
scaler = StandardScaler()
features_std = scaler.fit_transform(features)

# Caculate variance of each feature
selector = VarianceThreshold()
selector.fit(features_std).variances_

array([1., 1., 1., 1.])

## Thresholding Binary Feature Variance

Just like with numerical features, one strategy for selecting highly informative categorical
features is to examine their **variances**. In binary features (i.e., Bernoulli random
variables), variance is calculated as:

<font size='5'> $ Var x = p (1 − p) $ </font>


where $p$ is the proportion of observations of class 1. Therefore, by setting $p$, we can
remove features where the vast majority of observations are one class.

In [ ]:
## You have a set of binary categorical features and want to remove those with low variance (i.e., likely containing little information).
## S/ Select a subset of features with a Bernoulli random variable variance above a given threshold:

# Load library
from sklearn.feature_selection import VarianceThreshold
# Create feature matrix with:
# Feature 0: 80% class 0
# Feature 1: 80% class 1
# Feature 2: 60% class 0, 40% class 1

features = [[0, 1, 0],
            [0, 1, 1],
            [0, 1, 0],
            [0, 1, 1],
            [1, 0, 0]]

# Run threshold by variance
thresholder = VarianceThreshold(threshold=(.75 * (1 - .75)))
f_b = thresholder.fit_transform(features)

print(np.asarray(features).shape)
print(f_b.shape)

(5, 3)
(5, 1)


## Handling Highly Correlated Features

One problem we often run into in machine learning is **highly correlated features**. If
two features are highly correlated, then the information they contain is very similar,
and it is likely redundant to include both features. The solution to highly correlated
features is simple: remove one of them from the feature set.

In [ ]:
## You have a feature matrix and suspect some features are highly correlated.
## S/ Use a correlation matrix to check for highly correlated features. 
## If highly correlated features exist, consider dropping one of the correlated features:

# Load libraries
import pandas as pd
import numpy as np

# Create feature matrix with two highly correlated features
features = np.array([[1, 1, 1],
                     [2, 2, 0],
                     [3, 3, 1],
                     [4, 4, 0],
                     [5, 5, 1],
                     [6, 6, 0],
                     [7, 7, 1],
                     [8, 7, 0],
                     [9, 7, 1]])

# Convert feature matrix into DataFrame
dataframe = pd.DataFrame(features)

# Create correlation matrix
corr_matrix = dataframe.corr().abs()

# Select upper triangle of correlation matrix
upper = corr_matrix.where(np.triu(np.ones(corr_matrix.shape),
k=1).astype(np.bool))

# Find index of feature columns with correlation greater than 0.95
to_drop = [column for column in upper.columns if any(upper[column] > 0.95)]

print(to_drop)

[1]


<ipython-input-21-52863aeaedf4>:28: DeprecationWarning: `np.bool` is a deprecated alias for the builtin `bool`. To silence this warning, use `bool` by itself. Doing this will not modify any behavior and is safe. If you specifically wanted the numpy scalar type, use `np.bool_` here.
Deprecated in NumPy 1.20; for more details and guidance: https://numpy.org/devdocs/release/1.20.0-notes.html#deprecations
  k=1).astype(np.bool))


In [ ]:
# Drop features
dataframe.drop(dataframe.columns[to_drop], axis=1).head(3)

,0,2
0,1,1
1,2,0
2,3,1


In our solution, first we create a correlation matrix of all features:

In [ ]:
# Correlation matrix
dataframe.corr()

,0,1,2
0,1.000000,0.976103,0.000000
1,0.976103,1.000000,-0.034503
2,0.000000,-0.034503,1.000000


Second, we look at the upper triangle of the correlation matrix to identify pairs of highly correlated features:

In [ ]:
# Upper triangle of correlation matrix
upper

,0,1,2
0,NaN,0.976103,0.000000
1,NaN,NaN,0.034503
2,NaN,NaN,NaN


Third, we remove one feature from each of those pairs from the feature set.

## Removing Irrelevant Features for Classification

**Chi-square statistics** examines the independence of two categorical vectors. That is, the statistic is the difference between the observed number of observations in each
class of a categorical feature and what we would expect if that feature was independent
(i.e., no relationship) with the target vector:

<font size='5'> $ \chi^2=\sum_{i=1}^{n} \frac{(O_i - E_i)^2}{E_i} $ </font>

where $O_i$ is the number of observations in class $i$ and $E_i$ is the number of observations
in class $i$ we would expect if there is no relationship between the feature and target
vector.

In [ ]:
## You have a categorical target vector and want to remove uninformative features.
## S/ If the features are categorical, calculate a chi-square (χ2) statistic between each feature and the target vector:

# Load libraries
from sklearn.datasets import load_iris
from sklearn.feature_selection import SelectKBest
from sklearn.feature_selection import chi2, f_classif

# Load data
iris = load_iris()
features = iris.data
target = iris.target

# Convert to categorical data by converting data to integers
features = features.astype(int)

# Select two features with highest chi-squared statistics
chi2_selector = SelectKBest(chi2, k=2)
features_kbest = chi2_selector.fit_transform(features, target)

# Show results
print("Original number of features:", features.shape[1])
print("Reduced number of features:", features_kbest.shape[1])

Original number of features: 4
Reduced number of features: 2


If the features are quantitative, compute the ANOVA F-value between each feature and the target vector:

In [ ]:
# Select two features with highest F-values
fvalue_selector = SelectKBest(f_classif, k=2)
features_kbest = fvalue_selector.fit_transform(features, target)

# Show results
print("Original number of features:", features.shape[1])
print("Reduced number of features:", features_kbest.shape[1])

Original number of features: 4
Reduced number of features: 2


Instead of selecting a specific number of features, we can also use `SelectPercentile` to select the top `n` percent of features:

In [ ]:
# Load library
from sklearn.feature_selection import SelectPercentile

# Select top 75% of features with highest F-values
fvalue_selector = SelectPercentile(f_classif, percentile=75)
features_kbest = fvalue_selector.fit_transform(features, target)

# Show results
print("Original number of features:", features.shape[1])
print("Reduced number of features:", features_kbest.shape[1])

Original number of features: 4
Reduced number of features: 3


By calculating the chi-squared statistic between a
feature and the target vector, we obtain a measurement of the independence between the two. 
- If the target is independent of the feature variable, then it is irrelevant for our purposes because it contains no information we can use for classification. 
- If the two features are highly dependent, they likely are very informative for training our model.

## Recursively Eliminating Features

The idea behind RFE is to train a model that contains some parameters (also called *weights* or *coefficients*) like linear regression or support vector machines repeatedly.
The first time we train the model, we include all the features. Then, we find the feature with the smallest parameter (notice that this assumes the features are either rescaled or standardized), meaning it is less important, and remove the feature from the feature set.

We can use cross-validation (CV) to find the optimum number of features to keep during RFE. Specifically, in RFE with CV after every iteration, we use cross-validation to evaluate our
model. If CV shows that our model improved after we eliminated a feature, then we continue on to the next loop. However, if CV shows that our model got worse after we eliminated a feature, we put that feature back into the feature set and select those
features as the best.

https://scikit-learn.org/stable/auto_examples/feature_selection/plot_rfe_with_cross_validation.html#sphx-glr-auto-examples-feature-selection-plot-rfe-with-cross-validation-py

In [ ]:
## You want to automatically select the best features to keep.
## S/ Use scikit-learn’s RFECV to conduct recursive feature elimination (RFE) using crossvalidation (CV). 
##    That is, repeatedly train a model, each time removing a feature until model performance 
##    (e.g., accuracy) becomes worse. The remaining features are the best:

# Load libraries
import warnings
from sklearn.datasets import make_regression
from sklearn.feature_selection import RFECV
from sklearn import datasets, linear_model

# Suppress an annoying but harmless warning
warnings.filterwarnings(action="ignore", module="scipy",
message="^internal gelsd")

# Generate features matrix, target vector, and the true coefficients
features, target = make_regression(n_samples = 10000,
                                   n_features = 100,
                                   n_informative = 2,
                                   random_state = 1)

# Create a linear regression
ols = linear_model.LinearRegression()

# Recursively eliminate features
rfecv = RFECV(estimator=ols, step=1, scoring="neg_mean_squared_error")
rfecv.fit(features, target).transform(features)

array([[ 0.00850799,  0.7031277 ],
       [-1.07500204,  2.56148527],
       [ 1.37940721, -1.77039484],
       ...,
       [-0.80331656, -1.60648007],
       [ 0.39508844, -1.34564911],
       [-0.55383035,  0.82880112]])

Once we have conducted RFE, we can see the number of features we should keep:

In [ ]:
# Number of best features
rfecv.n_features_

2

We can also see which of those features we should keep:

In [ ]:
# Which categories are best
rfecv.support_

array([False, False, False, False, False,  True, False, False, False,
       False, False, False, False, False, False, False, False, False,
       False, False, False, False, False, False, False, False, False,
       False, False, False, False, False, False, False, False, False,
       False, False, False,  True, False, False, False, False, False,
       False, False, False, False, False, False, False, False, False,
       False, False, False, False, False, False, False, False, False,
       False, False, False, False, False, False, False, False, False,
       False, False, False, False, False, False, False, False, False,
       False, False, False, False, False, False, False, False, False,
       False, False, False, False, False, False, False, False, False,
       False])

We can even view the rankings of the features:

In [ ]:
# Rank features best (1) to worst
rfecv.ranking_

array([33, 39, 42, 20,  6,  1, 82, 35, 32,  3, 10, 72, 24, 44, 12, 49, 93,
       84, 94,  2, 25, 21, 78, 31, 43, 50, 47, 52, 81, 23, 61, 96, 80, 14,
       15, 58, 75, 29, 83,  1, 18, 68, 46, 19, 30,  5, 48, 60, 56, 69, 89,
        4, 79, 62, 11,  7, 98, 17, 71, 95, 54, 65,  9, 77, 53, 67, 16, 87,
       41, 85, 97, 70, 26, 76, 59, 99, 36, 34, 38, 90, 55, 64, 57, 88, 22,
       73, 86, 92, 27, 51, 66, 13, 74, 45, 40, 63, 37, 28,  8, 91])